<a href="https://colab.research.google.com/github/QaziSaim/DeepLearning--DeepNeuralNetwork-/blob/main/PyTroch_NeuralNetworkOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
import torchsummary

In [4]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target

# train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

from keras.utils import to_categorical
y_train = to_categorical(y_train,3)
y_test = to_categorical(y_test,3)

In [8]:
X_train.shape

(105, 4)

In [17]:
class MyModel(nn.Module):
  def __init__(self, input_dim):
    super(MyModel, self).__init__()
    self.fc1 = nn.Linear(input_dim, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 3)
    self.relu = nn.ReLU()
    # self.softmax = nn.Softmax()
    # self.softmax = nn.Softmax()

  def forward(self, x):
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.fc3(x)
    return x
# Initilization model
input_dim = X_train.shape[1]
model = MyModel(input_dim)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

x_train_tensor = torch.tensor(X_train,dtype = torch.float32)
y_train_tensor = torch.tensor(y_train.argmax(axis=1),dtype = torch.long)

epochs = 50
for epoch in range(epochs):
    # Forward pass
    outputs = model(x_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == y_train_tensor).sum().item()
        accuracy = correct / y_train_tensor.size(0)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.argmax(axis=1), dtype=torch.long)

with torch.no_grad():
    outputs = model(x_test_tensor)
    probs = torch.softmax(outputs, dim=1)
    _, predicted = torch.max(probs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    test_acc = correct / y_test_tensor.size(0)
    print("Test Accuracy:", test_acc)



Epoch [10/50], Loss: 0.9310, Accuracy: 0.7524
Epoch [20/50], Loss: 0.8106, Accuracy: 0.8190
Epoch [30/50], Loss: 0.6873, Accuracy: 0.8190
Epoch [40/50], Loss: 0.5683, Accuracy: 0.8286
Epoch [50/50], Loss: 0.4667, Accuracy: 0.8381
Test Accuracy: 0.8444444444444444


In [18]:
from torchsummary import summary
summary(model, input_size=(32, input_dim))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 32, 64]             320
              ReLU-2               [-1, 32, 64]               0
            Linear-3               [-1, 32, 32]           2,080
              ReLU-4               [-1, 32, 32]               0
            Linear-5                [-1, 32, 3]              99
Total params: 2,499
Trainable params: 2,499
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.01
Estimated Total Size (MB): 0.06
----------------------------------------------------------------
